# PINN for flow around a cylinder
The Navier-Stokes equations are being solved for incompressible flow around a cylinder. These equations are a set of partial differential equations (PDEs) governing the motion of fluid substances. For a 2D incompressible, viscous flow, they can be written in the Cartesian coordinate system as follows:

## Continuity Equation:

The continuity equation represents the conservation of mass, stating that the mass entering a control volume must equal the mass exiting the volume, plus any change in mass within the volume. For an incompressible flow, it simplifies to:

$$ \nabla u = 0 $$

or, in a two-dimensional form:

$$
\begin{equation}
\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0
\tag{1}
\end{equation}
$$

where u and v are the velocity components in the x and y directions respectively.

## Momentum Equations:

The momentum equations represent the conservation of momentum. They are the result of applying Newton's second law (force equals the rate of change of momentum) to fluid motion. The Navier-Stokes equations include the effects of viscosity, which are modeled with a Laplacian operator. The momentum equations for an incompressible flow can be written as:

$$
\begin{equation}
\rho \left( \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} \right) = -\frac{\partial p}{\partial x} + \nu \nabla^2 u
\tag{2}
\end{equation}
$$
$$
\begin{equation}
\rho \left( \frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} \right) = -\frac{\partial p}{\partial y} + \nu \nabla^2 v
\tag{3}
\end{equation}
$$

where:

ρ is the fluid density
u, v are the velocity components in the x and y directions, respectively
t is time
p is pressure
ν is the kinematic viscosity
∇² is the Laplacian operator, denoting the divergence of the gradient of a scalar field, here used to represent the diffusion of momentum (due to viscosity)
The pressure-Poisson equation is also used to enforce incompressibility:

$$
\begin{equation}
\nabla^2 p = -\rho \left[ \left( \frac{\partial (u^2)}{\partial x} + \frac{\partial (uv)}{\partial y} \right) + \left( \frac{\partial (uv)}{\partial x} + \frac{\partial (v^2)}{\partial y} \right) \right]
\tag{4}
\end{equation}
$$
where
$$
\nabla^2 p = \frac{\partial^2 p}{\partial x^2} + \frac{\partial^2 p}{\partial y^2}
$$

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

import math
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"

In [ ]:
xmax = 1.0
ymax = 1.0
tmax = 0.1

In [ ]:
rho = 1
mu = .1

In [ ]:
# Define the cylinder
center = [0.5, 0.5]  # center of the cylinder
radius = 0.1  # radius of the cylinder

In [ ]:
def compute_grad(x, y):
        return torch.autograd.grad(x, y, grad_outputs=torch.ones_like(x), create_graph=True, retain_graph=True, only_inputs=True)[0]

In [ ]:
def get_collocation_points(xmin, xmax, ymin, ymax, tmin, tmax, center, radius, n):
    
    x = []
    y = []
    t = []

    h, k = center  # center of the circle

    while len(x) < n:  # generate Nf points
        # Generate random point in the rectangle
        x_point = xmin + (xmax - xmin) * random.random()
        y_point = ymin + (ymax - ymin) * random.random()
        t_point = tmin + (tmax - tmin) * random.random()
        
        # Check if point is outside the circle
        if (x_point - h) ** 2 + (y_point - k) ** 2 >= radius ** 2:
            x.append(x_point)
            y.append(y_point)
            t.append(t_point)
    return x, y, t

def get_boundary_points(xmin, xmax, tmin, tmax, n):
    
    x = []
    t = []

    while len(x) < n:  # generate Nb points
    # Generate random point in the rectangle
        x_point = xmin + (xmax - xmin) * random.random()
        t_point = tmin + (tmax - tmin) * random.random()
        
        x.append(x_point)
        t.append(t_point)

    return x, t

def get_cylinder_bc_points(center, radius, tmin, tmax, n):

    x_center, y_center = center
    x = []
    y = []
    t = []
    
    for _ in range(n):
        # Generate random angle and random radius
        rand_angle = 2 * math.pi * random.random()
        rand_radius = radius * math.sqrt(random.random())
        # Convert polar coordinates to cartesian
        x_point = x_center + rand_radius * math.cos(rand_angle)
        y_point = y_center + rand_radius * math.sin(rand_angle)
        t_point = tmin + (tmax - tmin) * random.random()

        # Append to the lists
        x.append(x_point)
        y.append(y_point)
        t.append(t_point)

    return x, y, t

def get_initial_points(xmin, xmax, ymin, ymax, n):

    x = []
    y = []

    while len(x) < n:
    # Generate random point in the rectangle
        x_point = xmin + (xmax - xmin) * random.random()
        y_point = ymin + (ymax - ymin) * random.random()
        
        x.append(x_point)
        y.append(y_point)

    return x, y

In [ ]:
import os
import pandas as pd
import numpy as np

def get_u_data(indices_to_extract, dir_path, Nt):

    # Dictionary to store the results
    result_dict = {}

    for time_step in range(Nt):
        file_path = dir_path + "/u_" + str(time_step) + ".csv"
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, header=None)
            result_dict[file_path] = [df.iloc[i, j] for i, j in indices_to_extract]
        else: 
            print("error")
            exit()

    u_data = []

    for file_path, values in result_dict.items():
        u_data.append(values)

    return np.array(u_data)

def get_v_data(indices_to_extract, dir_path, Nt):

    # Dictionary to store the results
    result_dict = {}

    for time_step in range(Nt):
        file_path = dir_path + "/v_" + str(time_step) + ".csv"
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, header=None)
            result_dict[file_path] = [df.iloc[i, j] for i, j in indices_to_extract]
        else: 
            print("error")
            exit()

    v_data = []

    for file_path, values in result_dict.items():
        v_data.append(values)

    return np.array(v_data)

def get_p_data(indices_to_extract, dir_path, Nt):

    # Dictionary to store the results
    result_dict = {}

    for time_step in range(Nt):
        file_path = dir_path + "/p_" + str(time_step) + ".csv"
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, header=None)
            result_dict[file_path] = [df.iloc[i, j] for i, j in indices_to_extract]
        else: 
            print("error")
            exit()

    p_data = []

    for file_path, values in result_dict.items():
        p_data.append(values)

    return np.array(p_data)

def get_data_coords(data_indeces, dx, dy):

    coords = []

    for x_index, y_index in data_indeces:
        coords.append([x_index * dx, y_index * dy])

    return coords

def get_timestamps(Nt, dt):

    timestamps = []

    for timestep in range(Nt):
        timestamps.append(timestep * dt)

    return np.array(timestamps)

def get_training_data_batch(all_training_inputs, all_training_outputs, Nu):

    selected_indeces = random.sample(range(len(all_training_inputs)), Nu)

    inputs = all_training_inputs[selected_indeces, :]
    outputs = all_training_outputs[selected_indeces, :]

    x = inputs[:, 0]
    y = inputs[:, 1]
    t = inputs[:, 2]

    u = outputs[:, 0]
    v = outputs[:, 1]
    p = outputs[:, 2]

    return x, y, t, u, v, p

In [ ]:
# Specified indices to extract
# 100 points * 1,000 timestamps = 100,000 samples
data_indeces = [(10, 50), (10, 10), (50, 50), (50, 10)] # 0-based

_x = 10
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

_x = 20
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

_x = 30
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

_x = 40
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

_x = 50
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

_x = 60
for _y in range(0, 64, 4):
    data_indeces.append((_x, _y))

# Directory containing the CSV files
u_data_dir_path = 'C:/Users/gitop/repos/pinns/src/python/cylinder_flow_2d/csv_files/u'
v_data_dir_path = 'C:/Users/gitop/repos/pinns/src/python/cylinder_flow_2d/csv_files/v'
p_data_dir_path = 'C:/Users/gitop/repos/pinns/src/python/cylinder_flow_2d/csv_files/p'

# Number of timesteps
Nt = 1000

xmax = 1.0
ymax = 1.0
tmax = 0.1

Nx = 64
Ny = 64
Nt = 1000

dx = xmax / (Nx - 1)
dy = ymax / (Ny - 1)
dt = tmax / (Nt - 1)

def load_training_data():

    u_data = get_u_data(data_indeces, u_data_dir_path, Nt)
    v_data = get_v_data(data_indeces, v_data_dir_path, Nt)
    p_data = get_p_data(data_indeces, p_data_dir_path, Nt)

    # Compute means and standard deviations
    u_mean, u_std = u_data.mean(), u_data.std()
    v_mean, v_std = v_data.mean(), v_data.std()
    p_mean, p_std = p_data.mean(), p_data.std()

    # Normalize the data
    u_data_normalized = (u_data - u_mean) / u_std
    v_data_normalized = (v_data - v_mean) / v_std
    p_data_normalized = (p_data - p_mean) / p_std

    data_coords = get_data_coords(data_indeces, dx, dy)
    timestamps = get_timestamps(Nt, dt)

    inputs = []

    for t in timestamps:
        for x, y in data_coords:
            inputs.append([x, y, t])

    inputs = np.array(inputs)

    outputs = []

    for t in range(Nt):
        for i in range(len(data_coords)):
            outputs.append([u_data_normalized[t, i], v_data_normalized[t, i], p_data_normalized[t, i]])

    outputs = np.array(outputs)

    return inputs, outputs

In [ ]:
all_training_inputs, all_training_outputs = load_training_data()

In [ ]:
def get_batch(Nf, Nc, Nb, Nu, Ni):

    xf, yf, tf = get_collocation_points(0, xmax, 0, ymax, 0, tmax, center, radius, Nf)
    xc, yc, tc = get_cylinder_bc_points(center, radius, 0, tmax, Nc)

    xb_left = np.zeros(int(Nb/4))
    yb_left, tb_left = get_boundary_points(0, ymax, 0, tmax, int(Nb/4))

    xb_right = np.ones(int(Nb/4))
    yb_right, tb_right = get_boundary_points(0, ymax, 0, tmax, int(Nb/4))

    xb_up, tb_up = get_boundary_points(0, xmax, 0, tmax, int(Nb/4))
    yb_up = np.ones(int(Nb/4))

    xb_down, tb_down = get_boundary_points(0, xmax, 0, tmax, int(Nb/4))
    yb_down = np.zeros(int(Nb/4))

    xu, yu, tu, uu, vu, pu = get_training_data_batch(all_training_inputs, all_training_outputs, Nu)

    xic, yic = get_initial_points(0, xmax, 0, ymax, Ni)

    # Convert numpy arrays to PyTorch tensors
    xf_tensor = torch.tensor(xf, dtype=torch.float32, device=device, requires_grad=True)
    yf_tensor = torch.tensor(yf, dtype=torch.float32, device=device, requires_grad=True)
    tf_tensor = torch.tensor(tf, dtype=torch.float32, device=device, requires_grad=True)

    xc_tensor = torch.tensor(xc, dtype=torch.float32, device=device, requires_grad=True)
    yc_tensor = torch.tensor(yc, dtype=torch.float32, device=device, requires_grad=True)
    tc_tensor = torch.tensor(tc, dtype=torch.float32, device=device, requires_grad=True)

    xb_left_tensor = torch.tensor(xb_left, dtype=torch.float32, device=device, requires_grad=True)
    yb_left_tensor = torch.tensor(yb_left, dtype=torch.float32, device=device, requires_grad=True)
    tb_left_tensor = torch.tensor(tb_left, dtype=torch.float32, device=device, requires_grad=True)

    xb_right_tensor = torch.tensor(xb_right, dtype=torch.float32, device=device, requires_grad=True)
    yb_right_tensor = torch.tensor(yb_right, dtype=torch.float32, device=device, requires_grad=True)
    tb_right_tensor = torch.tensor(tb_right, dtype=torch.float32, device=device, requires_grad=True)

    xb_up_tensor = torch.tensor(xb_up, dtype=torch.float32, device=device, requires_grad=True)
    yb_up_tensor = torch.tensor(yb_up, dtype=torch.float32, device=device, requires_grad=True)
    tb_up_tensor = torch.tensor(tb_up, dtype=torch.float32, device=device, requires_grad=True)

    xb_down_tensor = torch.tensor(xb_down, dtype=torch.float32, device=device, requires_grad=True)
    yb_down_tensor = torch.tensor(yb_down, dtype=torch.float32, device=device, requires_grad=True)
    tb_down_tensor = torch.tensor(tb_down, dtype=torch.float32, device=device, requires_grad=True)

    xu_tensor = torch.tensor(xu, dtype=torch.float32, device=device, requires_grad=True)
    yu_tensor = torch.tensor(yu, dtype=torch.float32, device=device, requires_grad=True)
    tu_tensor = torch.tensor(tu, dtype=torch.float32, device=device, requires_grad=True)

    uu_tensor = torch.tensor(uu, dtype=torch.float32, device=device, requires_grad=True)
    vu_tensor = torch.tensor(vu, dtype=torch.float32, device=device, requires_grad=True)
    pu_tensor = torch.tensor(pu, dtype=torch.float32, device=device, requires_grad=True)

    xic_tensor = torch.tensor(xic, dtype=torch.float32, device=device, requires_grad=True)
    yic_tensor = torch.tensor(yic, dtype=torch.float32, device=device, requires_grad=True)

    return xf_tensor, yf_tensor, tf_tensor, xc_tensor, yc_tensor, tc_tensor, xb_left_tensor, yb_left_tensor, tb_left_tensor, xb_right_tensor, yb_right_tensor, tb_right_tensor, xb_up_tensor, yb_up_tensor, tb_up_tensor, xb_down_tensor, yb_down_tensor, tb_down_tensor, xu_tensor, yu_tensor, tu_tensor, uu_tensor, vu_tensor, pu_tensor, xic_tensor, yic_tensor

In [ ]:
# Nf = 1000
# Nc = 4000
# Nb = 4000
# Nu = 5000
# Ni = 4000

In [ ]:
import plotly.graph_objects as go

def plot_batch():

    xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down, xic, yic = map(lambda x: x.detach(), get_batch(Nf, Nc, Nb, Nu, Ni))

    xb = np.concatenate((xb_left.detach().numpy(), xb_right.detach().numpy(), xb_up.detach().numpy(), xb_down.detach().numpy()))
    yb = np.concatenate((yb_left.detach().numpy(), yb_right.detach().numpy(), yb_up.detach().numpy(), yb_down.detach().numpy()))
    tb = np.concatenate((tb_left.detach().numpy(), tb_right.detach().numpy(), tb_up.detach().numpy(), tb_down.detach().numpy()))

    fig1 = go.Figure(
        data = [
                go.Scatter3d(
                    x=xf,
                    y=tf,
                    z=yf,
                    mode='markers',
                    marker=dict(
                        size=2,
                        color="blue",
                        colorscale='Viridis',
                        opacity=0.8),
                    name="Collocation points",
                    showlegend=True)])

    fig2 = go.Figure(
        data = [
                go.Scatter3d(
                    x=xb,
                    y=tb,
                    z=yb,
                    mode='markers',
                    marker=dict(
                        size=2,
                        color="green",
                        colorscale='Viridis',
                        opacity=0.8),
                        name="Boundary points",
                        showlegend=True),
                go.Scatter3d(
                    x=xc,
                    y=tc,
                    z=yc,
                    mode='markers',
                    marker=dict(
                        size=2,
                        color="red",
                        colorscale='Viridis',
                        opacity=0.8),
                    name="Cylinder boundary points",
                    showlegend=True),
                go.Scatter3d(
                    x=xic,
                    y=np.zeros_like(xic),
                    z=yic,
                    mode='markers',
                    marker=dict(
                        size=2,
                        color="yellow",
                        colorscale='Viridis',
                        opacity=0.8),
                    name="Initial points",
                    showlegend=True)
                    ])

    # Updating titles and labels
    for fig in [fig1, fig2]:
        fig.update_layout(
            scene=dict(
                xaxis_title='x',
                yaxis_title='t',
                zaxis_title='y'
            ),
            width=700,
            margin=dict(r=20, b=10, l=10, t=10))
        # Show the plot
        fig.show()

# plot_batch()

## Fully Connected Neural Network with Skip Connections (FCNNskip)

Note that the skip connection layers are created in parallel with the regular layers. In the forward pass, we add the output of the skip connection to the output of the regular layer, following the standard practice for implementing skip connections.

In the loop that creates the layers, we create an extra "skip" layer every 2 layers, starting from the first layer. This layer has the same output size as the corresponding regular layer but always has an input size of 3 (the size of the input to the network). This is because the skip connections "skip" over the intervening layers and always connect back to the original input.

In the forward pass, we create a separate skip_input tensor that we pass through the skip layers. This is because the regular input tensor is being modified by the regular layers, but we want the skip connections to always connect back to the original input.

Then, every 2 layers, we add the output of the skip layer to the output of the regular layer before passing it through the activation function. This is the key feature of skip connections: they allow the network to learn an "identity" function that can bypass the intervening layers if necessary.

Finally, note that the output layer does not have a corresponding skip layer, since it's the final layer in the network.

In [ ]:
import torch.nn.init as init

class PINN(nn.Module):

    def __init__(self, hidden_units):

        super(PINN, self).__init__()
        
        self.layers = nn.ModuleList()
        self.skip_layers = nn.ModuleList()  # for skip connections
        in_units = 3
        out_units = 3
        
        for i, units in enumerate(hidden_units):
            layer = nn.Sequential(
                nn.Linear(in_units, units),
                nn.BatchNorm1d(units),
                nn.Tanh()
            )
            self.layers.append(layer)
            in_units = units
            # Create additional layers for skip connections
            if i % 2 == 0:
                skip_layer = nn.Sequential(
                    nn.Linear(3, units),  # from input dimension to current hidden units
                    nn.BatchNorm1d(units),
                )
                self.skip_layers.append(skip_layer)

        output_layer = nn.Sequential(
            nn.Linear(in_units, out_units),
            nn.BatchNorm1d(out_units),
        )
        self.layers.append(output_layer)

        # Initialize layers with Xavier initialization
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x, y, t):

        x = x.flatten()
        y = y.flatten()
        t = t.flatten()

        input = torch.stack([x, y, t], dim=-1)
        skip_input = input.clone()

        for i, layer in enumerate(self.layers[:-1]):
            output = layer(input)
            if i % 2 == 0 and i // 2 < len(self.skip_layers):  # apply skip connection every 2 layers
                skip_output = self.skip_layers[i // 2](skip_input)
                output = output + skip_output  # add skip connection output
            input = output
        output = self.layers[-1](input)
        u = output[:, 0]
        v = output[:, 1]
        p = output[:, 2]

        return u, v, p

    def mse_f(self, x, y, t):

        u, v, p = self(x, y, t)

        # Compute derivatives of u
        u_t = compute_grad(u, t)
        u_x = compute_grad(u, x)
        u_y = compute_grad(u, y)
        u_xx = compute_grad(u_x, x)
        u_yy = compute_grad(u_y, y)

        # Compute derivatives of v
        v_t = compute_grad(v, t)
        v_x = compute_grad(v, x)
        v_y = compute_grad(v, y)
        v_xx = compute_grad(v_x, x)
        v_yy = compute_grad(v_y, y)

        # Compute derivatives of p
        p_x = compute_grad(p, x)
        p_xx = compute_grad(p_x, x)
        p_y = compute_grad(p, y)
        p_yy = compute_grad(p_y, y)

        # Compute additional derivatives
        u2_x = compute_grad(u**2, x)
        v2_y = compute_grad(v**2, y)
        uv_y = compute_grad(u*v, y)
        uv_x = compute_grad(u*v, x)

        f1 = u_x + v_y
        f2 = rho * (u_t + u*u_x + v*u_y) + p_x - mu * (u_xx + u_yy)
        f3 = rho * (v_t + u*v_x + v*v_y + p_y - mu * (v_xx + v_yy))
        # f4 = p_xx + p_yy + rho * (u_x**2 - 2*u_y*v_x - v_y**2)
        f4 = p_xx + p_yy + rho * (u2_x + uv_y + uv_x + v2_y)

        _mse = (1/4) *  (torch.mean(torch.square(f1)) + \
                        torch.mean(torch.square(f2)) + \
                        torch.mean(torch.square(f3)) + \
                        torch.mean(torch.square(f4)))

        return _mse

    def mse_ic(self, x, y):
        
        t = torch.zeros_like(x)
        u, v, p = self(x, y, t)

        _mse = torch.mean( torch.square(u) + torch.square(v) + torch.square(p) )

        return _mse

    def mse_cylinder_bc(self, x, y, t):

        u, v, p = self(x, y, t)

        _mse = torch.mean( torch.square(u) + torch.square(v) )

        return _mse

    def mse_left_bc(self, y, t):

        x = torch.zeros_like(y)
        u, v, p = self(x, y, t)

        _mse = torch.mean( torch.square(u - torch.ones_like(u)) + torch.square(v - torch.zeros_like(v)) )

        return _mse

    def mse_right_bc(self, y, t):

        x = torch.full_like(y, xmax)
        u, v, p = self(x, y, t)

        u_t = compute_grad(u, t)
        v_t = compute_grad(v, t)

        _mse = torch.mean( torch.square(u_t) + torch.square(v_t) )

        return _mse

    def mse_up_bc(self, x, t):

        y = torch.full_like(x, ymax)
        u, v, p = self(x, y, t)

        u_t = compute_grad(u, t)

        _mse = torch.mean( torch.square(u_t) + torch.square(v) )

        return _mse

    def mse_down_bc(self, x, t):

        y = torch.zeros_like(x)
        u, v, p = self(x, y, t)

        u_t = compute_grad(u, t)

        _mse = torch.mean( torch.square(u_t) + torch.square(v) )

        return _mse

    def mse_bc(self, yb_left, tb_left, yb_right, tb_right, xb_up, tb_up, xb_down, tb_down):

        _mse = (1/4) * (self.mse_left_bc(yb_left, tb_left) + self.mse_right_bc(yb_right, tb_right) + self.mse_up_bc(xb_up, tb_up) + self.mse_down_bc(xb_down, tb_down))

        return _mse

    def mse_u(self, xu, yu, tu, uu, vu, pu):

        u, v, p = self(xu, yu, tu)

        # _mse = (1/3) * (torch.mean(torch.square(uu - u)) + torch.mean(torch.square(vu - v)) + torch.mean(torch.square(pu - p)) )
        _mse = (1/2) * (torch.mean(torch.square(uu - u)) + torch.mean(torch.square(vu - v)) )

        return _mse

    def loss(self, cf, cc, cb, cu, ci, xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down, xu, yu, tu, uu, vu, pu, xic, yic):
        # unused values only for debugging

        _mse_f = self.mse_f(xf, yf, tf)
        _mse_cylinder_bc = self.mse_cylinder_bc(xc, yc, tc)
        _mse_bc = self.mse_bc(yb_left, tb_left, yb_right, tb_right, xb_up, tb_up, xb_down, tb_down)
        _mse_u = self.mse_u(xu, yu, tu, uu, vu, pu)
        _mse_ic = self.mse_ic(xic, yic)

        _loss = cf * _mse_f + cc * _mse_cylinder_bc +  cb * _mse_bc + cu * _mse_u + ci * _mse_ic
        # _loss = cc * _mse_cylinder_bc +  cb * _mse_bc + cu * _mse_u

        return _loss, _mse_f, _mse_cylinder_bc, _mse_bc, _mse_u, _mse_ic

    def set_training_params(self, optimizer, cf, cc, cb, cu, ci, Nf, Nc, Nb, Nu, Ni):

        self.optimizer = optimizer
        self.cf = cf
        self.cc = cc
        self.cb = cb
        self.cu = cu
        self.ci = ci
        self.Nf = Nf
        self.Nc = Nc
        self.Nb = Nb
        self.Nu = Nu
        self.Ni = Ni

    def closure(self):

        self.optimizer.zero_grad()

        xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down, xu, yu, tu, uu, vu, pu, xic, yic = get_batch(self.Nf, self.Nc, self.Nb, self.Nu, self.Ni)

        _loss, _, _, _, _, _ = self.loss(self.cf, self.cc, self.cb, self.cu, self.ci, xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down, xu, yu, tu, uu, vu, pu, xic, yic)
        _loss.backward()

        return _loss

    def report_loss(self):

        self.optimizer.zero_grad()

        xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down , xu, yu, tu, uu, vu, pu, xic, yic = get_batch(self.Nf, self.Nc, self.Nb, self.Nu, self.Ni)
        _loss, _mse_f, _mse_cylinder_bc, _mse_bc, _mse_u, _mse_ic = self.loss(self.cf, self.cc, self.cb, self.cu, self.ci, xf, yf, tf, xc, yc, tc, xb_left, yb_left, tb_left, xb_right, yb_right, tb_right, xb_up, yb_up, tb_up, xb_down, yb_down, tb_down, xu, yu, tu, uu, vu, pu, xic, yic)

        return _loss, _mse_f, _mse_cylinder_bc, _mse_bc, _mse_u, _mse_ic


    def train(self, epochs, optimizer, cf, cc, cb, cu, ci, Nf, Nc, Nb, Nu, Ni):

        self.set_training_params(optimizer, cf, cc, cb, cu, ci, Nf, Nc, Nb, Nu, Ni)

        self.training_inputs, self.training_outputs = load_training_data()

        for epoch in range(epochs):
            optimizer.step(self.closure)
            if epoch % 10 == 0:
                _loss, _mse_f, _mse_cylinder_bc, _mse_bc, _mse_u, _mse_ic = self.report_loss()
                print(f'Epoch: {epoch},\tTotal loss: {_loss},\tPDE loss: {_mse_f},\tCylinder BC loss: {_mse_cylinder_bc},\tBC loss: {_mse_bc},\tTraining data loss: {_mse_u},\tIC loss: {_mse_ic}')

Physical interpretation of the boundary conditions:

1. `mse_left_bc`: On the left boundary of the fluid domain (x=0), the fluid has a horizontal velocity `u` of 1, and a vertical velocity `v` of 0. This suggests a fluid entering the domain from the left with a unit speed and flowing only in the horizontal direction, i.e., no upward or downward movement of fluid on this boundary. 

2. `mse_right_bc`: On the right boundary of the fluid domain (x=xmax), the time derivatives of `u` and `v` are set to zero. This means the fluid's horizontal and vertical velocities aren't changing with time. This could imply either a steady-state flow or the fluid exiting the domain without any acceleration or deceleration at the right boundary.

3. `mse_up_bc`: On the top boundary of the fluid domain (y=ymax), the vertical velocity `v` is zero, meaning there's no fluid flowing out of or into the domain from the top. Additionally, the time derivative of the horizontal velocity `u` is zero, implying the horizontal movement of the fluid at this boundary is not changing over time.

4. `mse_down_bc`: On the bottom boundary of the fluid domain (y=0), similar to `mse_up_bc`, the vertical velocity `v` is zero, implying no fluid flow into or out of the domain from the bottom. Also, the time derivative of the horizontal velocity `u` is zero, suggesting no change in horizontal movement over time at the bottom boundary.

In short, these boundary conditions imply that fluid enters the domain from the left, moves horizontally without any vertical movement at the top and bottom boundaries, and exits from the right boundary without changing speed over time. Also, there's no fluid movement into or out of the domain from the top or bottom boundaries.

In [ ]:
# hidden_units = [32, 32, 32]
# hidden_units = [64, 64, 64, 64]
# hidden_units = [128, 128, 128, 128]
# hidden_units = [256, 256, 256, 256]
# hidden_units = [512, 512]
# hidden_units = [1024, 1024, 1024]
# hidden_units = [20, 40, 80, 100, 100, 80, 40, 20]
hidden_units = [128 for _ in range(20)]
# hidden_units = [20, 20, 20, 20, 20, 20, 20, 20]
# hidden_units = [100, 100, 100, 100, 100, 100]
# hidden_units = [64 for _ in range(4)]

pinn = PINN(hidden_units).to(device)

In [ ]:
import torch.optim as optim

optimizer = torch.optim.Adam(pinn.parameters(), lr=0.001)
# optimizer = torch.optim.LBFGS(pinn.parameters())
# optimizer = torch.optim.LBFGS(pinn.parameters(), 
#                               lr=1, 
#                               max_iter=50000, 
#                               max_eval=50000, 
#                               tolerance_grad=1.0 * np.finfo(float).eps, 
#                               tolerance_change=1.0 * np.finfo(float).eps, 
#                               history_size=50,
#                               line_search_fn='strong_wolfe')

epochs = 200

cf = 0.01
cc = 0.5
cb = 0.1
cu = 0.5
ci = 0.5

Nf = 1000
Nc = 4000
Nb = 4000
Nu = 5000
Ni = 4000

pinn.train(epochs, optimizer, cf, cc, cb, cu, ci, Nf, Nc, Nb, Nu, Ni)

In [ ]:
torch.save(pinn.state_dict(), './pinn4.pt')

In [ ]:
Nx = 100
Ny = 100

x_test = np.linspace(0, xmax, Nx)
y_test = np.linspace(0, ymax, Ny)
t_test = np.array([0.1])

x_test_grid, y_test_grid, t_test_grid = np.meshgrid(x_test, y_test, t_test)

# Evaluate the model at the grid points
x_test_tensor = torch.tensor(x_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
y_test_tensor = torch.tensor(y_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
t_test_tensor = torch.tensor(t_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)

with torch.no_grad():  # Disable gradient calculation to save memory and computation
    u, v, p = pinn(x_test_tensor, y_test_tensor, t_test_tensor)

In [ ]:
u = u.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])
v = v.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])
p = p.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors

fig, ax = plt.subplots()

velocity_magnitude = np.sqrt(u**2 + v**2)

x_test_grid_2d, y_test_grid_2d = np.meshgrid(x_test, y_test)

strm = ax.streamplot(x_test_grid_2d, y_test_grid_2d, u, v, color=velocity_magnitude, linewidth=1, cmap=cm.inferno)

ax.set_xlim(0, 1)  # Set x-axis limits
ax.set_ylim(0, 1)  # Set y-axis limits
ax.set_xlabel("x")  # Set y-axis limits
ax.set_ylabel("y")  # Set y-axis limits

norm = mcolors.Normalize(vmin=np.min(velocity_magnitude), vmax=np.max(velocity_magnitude))
sm = plt.cm.ScalarMappable(cmap=cm.inferno, norm=norm)
sm.set_array([])

fig.colorbar(sm, ax=ax, label='Velocity magnitude')

In [ ]:
_Nx = 100
_Ny = 100

x_test = np.linspace(0, xmax, _Nx)
y_test = np.linspace(0, ymax, _Ny)

u_all = np.array([])
v_all = np.array([])
p_all = np.array([])

_Nt = 20
_dt = tmax / (_Nt - 1)

for t in np.arange(0., tmax + _dt/2, _dt):
    _t = np.array([t])
    
    _x_test_grid, _y_test_grid, _t_test_grid = np.meshgrid(x_test, y_test, _t)

    _x_test_tensor = torch.tensor(_x_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
    _y_test_tensor = torch.tensor(_y_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)
    _t_test_tensor = torch.tensor(_t_test_grid.reshape(-1, 1), dtype=torch.float32, device=device, requires_grad=True)

    with torch.no_grad():  # Disable gradient calculation to save memory and computation
        _u, _v, _p = pinn(_x_test_tensor, _y_test_tensor, _t_test_tensor)
        _u = _u.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])
        _v = _v.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])
        _p = _p.detach().cpu().numpy().reshape(x_test.shape[0], y_test.shape[0])
        u_all = np.append(u_all, _u)
        v_all = np.append(v_all, _v)
        p_all = np.append(p_all, _p)

u_all = u_all.reshape(_Nt, _Nx, _Ny)
v_all = v_all.reshape(_Nt, _Nx, _Ny)
p_all = p_all.reshape(_Nt, _Nx, _Ny)

In [ ]:
_x_test_grid, _y_test_grid = np.meshgrid(x_test, y_test)

fig, ax = plt.subplots(figsize=(11, 7))

timestep = -1

# Compute the velocity magnitude
magnitude = np.sqrt(u_all[timestep]**2 + v_all[timestep]**2)

# Plotting the velocity magnitude field
contour = ax.contourf(_x_test_grid, _y_test_grid, magnitude, alpha=1, cmap=cm.plasma, levels=100)
fig.colorbar(contour, ax=ax, label='Velocity magnitude')

# Plotting the velocity field
quiver = ax.quiver(_x_test_grid[::4, ::4], _y_test_grid[::4, ::4], u_all[timestep][::4, ::4], v_all[timestep][::4, ::4])

# Plotting the cylinder
# ax.scatter(X[cylinder], Y[cylinder], color='black', s=100)

# Adding labels
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')

plt.show()

In [ ]:
import matplotlib.animation as animation
import matplotlib.colors as mcolors

fig, ax = plt.subplots(figsize=(11, 7))

# Adding labels
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')

# Initialization function for the animation
def init():
    return []

# Animation update function
def update(num, u_data, v_data, p_data, X, Y, cylinder):
    # Clear the current axes.
    plt.cla()

    # Compute the velocity magnitude for the current timestep
    magnitude = np.sqrt(u_data[num]**2 + v_data[num]**2)
    # magnitude = np.sqrt(p_data[num]**2)

    # Create a new contour plot for the velocity magnitude
    contour = ax.contourf(X, Y, magnitude, alpha=1, cmap=cm.plasma, levels=100)

    # Create a new quiver plot for the velocity field
    quiver = ax.quiver(X[::4, ::4], Y[::4, ::4], u_data[num][::4, ::4], v_data[num][::4, ::4])

    # Create a new scatter plot for the cylinder
    cylinder_plot = ax.scatter(X[cylinder], Y[cylinder], color='black', s=100)
    # cylinder_plot = ax.scatter()

    # Adding labels
    ax.set_xlabel('X Position')
    ax.set_ylabel('Y Position')

    return contour, quiver, cylinder_plot,

# Define the cylinder
center = [0.5, 0.5]  # center of the cylinder
radius = 0.1  # radius of the cylinder
cylinder = np.sqrt((_x_test_grid - center[0])**2 + (_y_test_grid - center[1])**2) < radius

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=range(len(u_all)), init_func=init, fargs=(u_all, v_all, p_all, _x_test_grid, _y_test_grid, cylinder), blit=False)

# Create a ScalarMappable object
norm = mcolors.Normalize(vmin=0, vmax=1)
sm = plt.cm.ScalarMappable(cmap=cm.inferno, norm=norm)
sm.set_array([])

# Add a colorbar to the figure using the ScalarMappable object
fig.colorbar(sm, ax=ax, label='Velocity magnitude')

# Save the animation
ani.save('flow_animation_pinn.gif', writer='pillow')